# P5-05: Results visualization - CRAG
- Used to display the prediction results
- Compares predicted masks with actual masks
- Compares the predicted objects with the actual objects 
- Illustration of Images, Masks and Glands

## Libraries, moduls

In [ ]:
# Imports
import numpy as np
from skimage.io import imread
import os
import matplotlib.pyplot as plt
import pickle # zum Speichern
import statistics # zur Auswertung
import scipy.stats as stat #+
# PL - Pinckaers, Litjens
from  MODULE.PL.metrics import ObjectDice, ObjectHausdorff, F1score
# JH - Jonas Heinke
from configuration_CRAG import Path   as PATH   # Paths and filenames
from configuration_CRAG import EXPERIMENT # Name / key of the experiment   
from MODULE.JH.img_array_transform import ArrayTransform as TRANSFORM
from MODULE.JH.visualize import Show as SHOW

In [ ]:
# For control
VERBOSE=True
# Experiment
print(EXPERIMENT)

## 1. List of file paths

In [ ]:
path=PATH() # Instance of the class required for method call
# Source paths
path_experiment   = path.results  / EXPERIMENT   #  Results of the experiment
path_inputImages       = path.results  / EXPERIMENT/ 'images'
path_actualMasks  = path.results  / EXPERIMENT/ 'actualMasks'
path_predictMasks = path.results  / EXPERIMENT/ 'predictMasks'
path_predictMasksMorph = path.results  / EXPERIMENT/ 'predictMasksMorph'
path_predictObjects= path.results  / EXPERIMENT/ 'predictObjects' 
path_predictContourCodes= path.results  / EXPERIMENT/ 'predictContourCodes'
#  Paths with filenames
inputImages_filenames  = np.sort(path.get_filenames(path_inputImages  , dateifilter= '*.png'))
actualMasks_filenames  = np.sort(path.get_filenames(path_actualMasks, dateifilter= '*.png'))
predictMasks_filenames = np.sort(path.get_filenames(path_predictMasks, dateifilter= '*.png'))
predictMasksMorph_filenames   = np.sort(path.get_filenames(path_predictMasksMorph, dateifilter= '*.png'))
predictObjects_filenames      = np.sort(path.get_filenames(path_predictObjects, dateifilter= '*.png'))
predictContourCodes_filenames = np.sort(path.get_filenames(path_predictContourCodes , dateifilter= '*.pkl'))
# Number of samples
sample_anzahl=len(inputImages_filenames) # -> the same for everyone

In [ ]:
if VERBOSE:
    print(path_experiment)

In [ ]:
print(f'Number of image-mask-samples for prediction: {sample_anzahl}\n')
if VERBOSE:
    for idx in range(len(inputImages_filenames)):
        print(idx, ' | ', os.path.basename(inputImages_filenames[idx]),' >',\
                          os.path.basename(actualMasks_filenames[idx]) ,' >',\
                          os.path.basename(predictMasks_filenames[idx]) ,' >',\
                          os.path.basename(predictMasksMorph_filenames[idx]) ,' >',\
                          os.path.basename(predictContourCodes_filenames[idx]))

## 2. Read in images, masks and contours
- The size of the test images and test masks correspond to those of the training.

In [ ]:
input_images=[]
actual_masks=[]
predict_masks=[]
predict_masks_morph=[]
predict_objects=[]
contourcodes_list=[]
for idx in range(sample_anzahl):
    input_images.append(imread(inputImages_filenames[idx]))
    actual_masks.append(imread(actualMasks_filenames[idx]))
    predict_masks.append(imread(predictMasks_filenames[idx]))
    predict_masks_morph.append(imread(predictMasksMorph_filenames[idx]))
    predict_objects.append(imread(predictObjects_filenames[idx]))
    file=open(predictContourCodes_filenames[idx], 'rb')
    contourcodes_list.append(pickle.load(file))   
if VERBOSE:    
    print(input_images[0].shape, actual_masks[0].shape, predict_masks[0].shape, predict_masks_morph[0].shape, predict_objects[0].shape)

## 3. Evaluation of the test set (MEASURE)

### a) Evaluation of the prediction masks without post-processing

In [ ]:
if VERBOSE:
    print(actual_masks[0])
    print(predict_masks[0])

In [ ]:
transform=TRANSFORM()
dice, hausdorff, f1, dice_full = 0, 0, 0, 0
i_error=0
i_leer=0
anzahl=0
dice_list=[]
protokolldatei = open(path_experiment / 'protocol_prediction_class-based.txt','w')
protokolldatei.write('---------------------------------------------\n')  #+
protokolldatei.write(f'images_res.shape: {input_images[0].shape},\
                    actual_masks_res.shape: {actual_masks[0].shape},\
                    predict_masks.shape: {predict_masks[0].shape}\n\n')
# Table header
print_string=(f' idx | dice-idx  | f1-score  | weighted shape | actual masks       | predict masks')
print(print_string)
protokolldatei.write(print_string+'\n')
for idx in range(sample_anzahl):
  try:
    if actual_masks[idx].max() != 0 and predict_masks[idx].max != 0:
      actual_mask= transform.twoClasses(actual_masks[idx])
      predict_mask= predict_masks[idx]
      dice_img = ObjectDice(predict_mask,        actual_mask)
      dice_list.append(dice_img) 
      f1_img = F1score(predict_mask,              actual_mask)
      hausdorff_img = ObjectHausdorff(predict_mask,  actual_mask)
      dice += dice_img
      f1 += f1_img
      hausdorff += hausdorff_img
      print_string=(f' {idx:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:14.3f} | {os.path.basename(actualMasks_filenames[idx])} | {os.path.basename(predictMasks_filenames[idx])}')
      print(print_string)
      anzahl += 1
      # --- Protokoll ---
      protokolldatei.write(print_string+'\n')
      #-------------------------------------
    else:
      i_leer += 1
      print('Leer: ',i_leer, 'Cycle', idx)   
  except:
    i_error += 1
    print('Error: ',i_error, 'Cycle', idx)

In [ ]:
title = '\n Prediction, class-based'
anzahl_emty_error = f'\n count io.: {anzahl} | empty: {i_leer} |  error: {i_error}'
kenngroessen = f'\n All result parameters (Average value of experiment) -> dice-idx: {dice / anzahl} | f1-score: {f1 / anzahl} | weighted shape: {hausdorff / anzahl}'
mittel_median = f'\n dice -> mittelwert: {np.mean(dice_list)} | Median: {statistics.median(dice_list)}'
standard_konvidenz = f'\n dice -> Standardabweichung S: {np.std(dice_list)} | Konfidenzintervalle mit t: {stat.t.interval(alpha=0.95, df=len(dice_list)-1, loc=np.mean(dice_list), scale=stat.sem(dice_list)) }' 
min_max_spannweite = f'\n dice -> Min: {min(dice_list)} | Max: {max(dice_list)} | Spannweite: {max(dice_list) - min(dice_list)}'
# Display
print(title)
print(anzahl_emty_error)
print(kenngroessen)
print(mittel_median ) 
print(standard_konvidenz)
print(min_max_spannweite)
# File
protokolldatei.write(title)
protokolldatei.write(anzahl_emty_error)
protokolldatei.write(kenngroessen)
protokolldatei.write(mittel_median)
protokolldatei.write(standard_konvidenz)
protokolldatei.write(min_max_spannweite)

protokolldatei.close() 

### b) Evaluation of the post-processing prediction masks

In [ ]:
dice, hausdorff, f1, dice_full = 0, 0, 0, 0
i_error=0
i_leer
anzahl=0
dice_list=[]

protokolldatei = open(path_experiment /\
                      'protocol_prediction_with_post-processing_class-base.txt','w')
protokolldatei.write('---------------------------------------------\n')  #+
protokolldatei.write(f'input_images_res.shape: {input_images[0].shape},\
                    actual_masks_res.shape: {actual_masks[0].shape},\
                    predict_masks_morph.shape: {predict_masks[0].shape}\n\n')
# Table header
print_string=(f' idx | dice-idx  | f1-score  | weighted shape | actual masks       | predict masks')
print(print_string)
protokolldatei.write(print_string+'\n')
for idx in range(sample_anzahl):
  try:
    if actual_masks[idx].max() != 0 and predict_masks_morph[idx].max != 0:
      # Compare objects with the masks of the same class
      actual_mask       = transform.twoClasses(actual_masks[idx])
      predict_mask_morph=predict_masks_morph[idx]
      dice_img = ObjectDice(predict_mask_morph,        actual_mask)
      dice_list.append(dice_img) 
      f1_img = F1score(predict_mask_morph,              actual_mask)
      hausdorff_img = ObjectHausdorff(predict_mask_morph,  actual_mask)
      dice += dice_img
      f1 += f1_img
      hausdorff += hausdorff_img
      print_string=(f' {idx:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:13.3f} | {os.path.basename(actualMasks_filenames[idx])} | {os.path.basename(predictMasksMorph_filenames[idx])}')
      print(print_string)
      anzahl +=1
      # --- Protokol ---
      protokolldatei.write(print_string+'\n')
      #-------------------------------------
    else:
      i_leer += 1
      print('Leer: ',i_leer, 'Cycle', idx)      
  except:
    i_error += 1
    print('Error: ',i_error, 'Cycle: ', idx)


In [ ]:
title = '\n Prediction with post-processing, class-based'
anzahl_emty_error = f'\n count io.: {anzahl} | empty: {i_leer} |  error: {i_error}'
kenngroessen = f'\n All result parameters (Average value of experiment) -> dice-idx: {dice / anzahl} | f1-score: {f1 / anzahl} | weighted shape: {hausdorff / anzahl}'
mittel_median = f'\n dice -> mittelwert: {np.mean(dice_list)} | Median: {statistics.median(dice_list)}'
standard_konvidenz = f'\n dice -> Standardabweichung S: {np.std(dice_list)} | Konfidenzintervalle mit t: {stat.t.interval(alpha=0.95, df=len(dice_list)-1, loc=np.mean(dice_list), scale=stat.sem(dice_list)) }' 
min_max_spannweite = f'\n dice -> Min: {min(dice_list)} | Max: {max(dice_list)} | Spannweite: {max(dice_list) - min(dice_list)}'
# Display
print(title)
print(anzahl_emty_error)
print(kenngroessen)
print(mittel_median ) 
print(standard_konvidenz)
print(min_max_spannweite)
# File
protokolldatei.write(title)
protokolldatei.write(anzahl_emty_error)
protokolldatei.write(kenngroessen)
protokolldatei.write(mittel_median)
protokolldatei.write(standard_konvidenz)
protokolldatei.write(min_max_spannweite)

protokolldatei.close() 

### c) Evaluation of the predicted glands (objects)

In [ ]:
# Bewerung der nachbearbeiten Prognosemasken
dice, hausdorff, f1, dice_full = 0, 0, 0, 0

i_error = 0
i_leer = 0
anzahl = 0
dice_list=[]

protokolldatei = open(path_experiment /\
                      'protokoll_prediction_with_post-processing_object-based.txt','w') #+
protokolldatei.write('---------------------------------------------\n')  #+
protokolldatei.write(f'images_res.shape: {input_images[0].shape},\
                    actual_masks_res.shape: {actual_masks[0].shape},\
                    predict_masks.shape: {predict_objects[0].shape}\n\n')
# Table header
print_string=(f' idx | dice-idx  | f1-score  | weighted shape | actual masks       | predict masks')
print(print_string)
protokolldatei.write(print_string+'\n')
for idx in range(sample_anzahl):
  try:
    if actual_masks[idx].max() != 0 and predict_objects[idx].max != 0: 
        # The objects (glands) of the mask end are compared.
        actual_mask = actual_masks[idx]
        predict_object = predict_objects[idx]
        dice_img = ObjectDice(predict_object,        actual_mask)
        dice_list.append(dice_img)                                      # new
        f1_img = F1score(predict_object,              actual_mask)
        hausdorff_img = ObjectHausdorff(predict_object,  actual_mask)
        dice += dice_img
        f1 += f1_img
        hausdorff += hausdorff_img
        print_string = (f' {idx:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:13.3f} | {os.path.basename(actualMasks_filenames[idx])} | {os.path.basename(predictObjects_filenames[idx])}')
        print(print_string)
        anzahl +=1
        # --- Protokol ---
        protokolldatei.write(print_string+'\n')
    else:
      i_leer += 1
      print('Leer: ',i_leer, 'Cycle', idx)      
  except:
    i_error += 1
    print('Error: ',i_error, 'Cycle: ', idx)

In [ ]:
title = '\n Prediction with post-processing, object-based'
anzahl_emty_error = f'\n count io.: {anzahl} | empty: {i_leer} |  error: {i_error}'
kenngroessen = f'\n All result parameters (Average value of experiment) -> dice-idx: {dice / anzahl} | f1-score: {f1 / anzahl} | weighted shape: {hausdorff / anzahl}'
mittel_median = f'\n dice -> mittelwert: {np.mean(dice_list)} | Median: {statistics.median(dice_list)}'
standard_konvidenz = f'\n dice -> Standardabweichung S: {np.std(dice_list)} | Konfidenzintervalle mit t: {stat.t.interval(alpha=0.95, df=len(dice_list)-1, loc=np.mean(dice_list), scale=stat.sem(dice_list)) }' 
min_max_spannweite = f'\n dice -> Min: {min(dice_list)} | Max: {max(dice_list)} | Spannweite: {max(dice_list) - min(dice_list)}'
# Display
print(title)
print(anzahl_emty_error)
print(kenngroessen)
print(mittel_median ) 
print(standard_konvidenz)
print(min_max_spannweite)
# File
protokolldatei.write(title)
protokolldatei.write(anzahl_emty_error)
protokolldatei.write(kenngroessen)
protokolldatei.write(mittel_median)
protokolldatei.write(standard_konvidenz)
protokolldatei.write(min_max_spannweite)

protokolldatei.close() 

In [ ]:
# Histogram of Dice
print(dice_list)
fig=plt.figure(figsize=(16,12))
n,bins,patches=plt.hist(dice_list, bins=11, range=(0.3, 0.9),histtype='bar',\
                        align='left', color='gray',alpha=0.5, density=True, cumulative=False, label='Dice-Index (Onjects)')
plt.xlabel('Dice-Index $Dice$', size=28)
plt.ylabel('Häufigkeit  $h($$\Delta$$P)$', size=28)
plt.title("Dice index histogram of the glands", size=28)
plt.tick_params(labelsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.savefig(path_experiment /'Dice index histogram of the glands')
plt.show()

## 4. Draw the contour in an array and in an image

In [ ]:
class DrawCodeInArray():
    ''' 2. Zeichnet in ein Array, hier eine Kontur
    HINWEIS: Schnellere Berechnung wenn Klasse im Notebook'''
    #Konstruktor
    def __init__(self, verbose):
        super(DrawCodeInArray, self).__init__()
        # Steuert Ausgabe zur Kontrolle    
        self.verbose = verbose
    def all_contours(self, contour_array, contour_codes, element_value=255, depth=1):
        '''
        2.2 Verwendet den Code um eine Kontur in eine Array zu übertragen.
        Eingang Methode:
            contour_array - Array, dass verändert wird
            contour_codes - Koordinaten der Konturen (mehrere pro array)
            element_value - Wert, den die Konturpunkte erhalten
            depth - Dicke des Konturzuges (Konturdicke)
        Rückgabe:
            contour_array- Array mit Kontur entsprechend des Konturcods
        '''
        for contour in contour_codes:
            for point in contour:
                #print(point[0], point[1]) #x, y
                contour_array[int(point[0]), int(point[1])]=element_value
                try:
                    for d in range(0, depth): # Kontour dicker zeichnen
                        contour_array[int(point[0]+d), int(point[1])+d]=element_value
                        contour_array[int(point[0]-d), int(point[1])-d]=element_value
                except:
                    pass
        return contour_array

In [ ]:
# -> contourcodes_list[0][0][1][0])
# [listenelement], [contour eines Listenelementes], [Punkt]  , [Punktkoordinaten_x_y]
# Contour as an array
contourarray_list=[]
for idx in range(sample_anzahl):
    # It is drawn in here
    contour_array=np.zeros(predict_objects[idx].shape, dtype=int)
    drawInArray=DrawCodeInArray(True)
    # Contour of a mask
    conturcodes=contourcodes_list[idx]
    print(f'{idx}', end=', ')
    contourarray=drawInArray.all_contours(contour_array,conturcodes, 255, 2)
    contourarray_list.append(contourarray)  

In [ ]:
# Draw the contour in the input image
contourimage_list=[]
for idx in range(sample_anzahl):
    contourimage=np.array(input_images[idx], copy=True)
    drawInArray=DrawCodeInArray(True)
    print(f'{idx}', end=', ')
    contourimage=drawInArray.all_contours(contourimage,contourcodes_list[idx], 0, 4)
    contourimage_list.append(contourimage) 

### Comparison of visualization

In [ ]:
show=SHOW(experiment=EXPERIMENT, figsize=(15,30), fontsize=16)

In [ ]:
idx_list=list([6,19,16])
path_set= path_experiment  / f'images_masks_listset_(P5-05)_{str(idx_list)}.png'
listset=list([input_images, actual_masks,         predict_masks,\
              predict_masks_morph,     predict_objects,  contourimage_list ] )
titles=list(['Image', 'Actual mask', 'predicted mask',\
             'post-processed',    'Predicted glands', 'Images with contours'])
show.list_set(idx_list, listset,titles, path=path_set)

In [ ]:
idx_list=list([10,20,26])
path_set= path_experiment  / f'images_masks_listset_(P05)_{str(idx_list)}.png'
listset=list([input_images, actual_masks,         predict_masks,\
              predict_masks_morph,     predict_objects,  contourimage_list ] )
titles=list(['Image', 'Actual mask', 'predicted mask',\
             'post-processed',    'Predicted glands', 'Images with contours'])
show.list_set(idx_list, listset,titles, path=path_set)

In [ ]:
print('End of visualizing')